In [1]:
!pip install transformers==4.41.1 datasets torch==2.1.0 mlflow==2.13.1 boto3==1.34.122 

In [2]:
from tqdm import tqdm
from datasets import load_metric
import torch
import mlflow
import mlflow.pytorch
from mlflow.models import infer_signature
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_
from src.utils import load_squad_data, SquadDataset
import os

In [3]:
# Set model training parameters
params = {
    'learning_rate': 5e-5,
    'batch_size': 16,
    'num_epochs': 3,
    'max_len': 512,
    'no_answer_threshold': 0.5,
    'tokenizer': "distilbert-base-uncased"
}

In [4]:
# Set environment variables for MLflow and Minio (object storage)
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "miniopass"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000" 
 
mlflow_tracking_url = "http://localhost:3000"
model_artifact_name = "bert_model"
experiment_name = "bert-baseline"

# Set up MLflow experiment tracking
mlflow.set_tracking_uri(mlflow_tracking_url)
mlflow.set_experiment(experiment_name)

# Select device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained(params['tokenizer'])
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")
model.to(device)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [5]:
# Load SQuAD training and validation datasets
train_data = load_squad_data('../data/train-v2.0.json')
dev_data = load_squad_data('../data/dev-v2.0.json')

# Create PyTorch Datasets
train_dataset = SquadDataset(train_data, tokenizer, params['max_len'])
dev_dataset = SquadDataset(dev_data, tokenizer, params['max_len'])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=params['batch_size'])

In [6]:
early_stopping_counter = 0

# Start MLflow run for logging experiments and metrics
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params(params)
    
    # Set up optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=params['learning_rate'])
    
    input_ids, attention_mask, outputs = None, None, None
    
    print("Training...")
    model.train()
    
    # Training loop for specified epochs
    for epoch in range(params['num_epochs']):
        total_loss = 0
        for batch in tqdm(train_loader, leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)
    
            # Zero out previous gradients
            optimizer.zero_grad() 
    
            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            loss = outputs.loss
    
            # Backward pass (gradient calculation)
            loss.backward()
            
            # Update model parameters
            optimizer.step()
    
            # Track total loss for the epoch
            total_loss += loss.item()
    
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")
    
    # Load evaluation metric (SQuAD v2 format)
    metric = load_metric("squad_v2")
    predictions = []
    references = []
    
    print("Evaluation...")
    model.eval()
    
    # Evaluation loop
    for batch in tqdm(dev_loader, leave=False):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
    
        # Disable gradient calculation for evaluation
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits
    
        # Get the predicted start and end positions
        start_pred = torch.argmax(start_logits, dim=-1)
        end_pred = torch.argmax(end_logits, dim=-1)
        
        # Calculate max start and end logits for no-answer probability
        start_logit_max = torch.max(start_logits, dim=-1).values
        end_logit_max = torch.max(end_logits, dim=-1).values
    
        # Calculate no-answer probability
        no_answer_prob = 1 - (start_logit_max + end_logit_max) / 2.0
    
        # Process batch predictions and references
        for i in range(input_ids.size(0)):
            # Convert start and end predictions to Python integers
            pred_start = start_pred[i].item()
            pred_end = end_pred[i].item()
    
            # Handle no-answer predictions based on threshold
            if no_answer_prob[i] > params['no_answer_threshold']:
                pred_answer = ""  # No answer prediction
            else:
                # Handle no-answer predictions based on threshold
                pred_answer = tokenizer.decode(input_ids[i][pred_start:pred_end + 1], skip_special_tokens=True)
    
            # Decode ground truth (reference) answer
            true_start = start_positions[i].item()
            true_end = end_positions[i].item()
            true_answer = tokenizer.decode(input_ids[i][true_start:true_end + 1], skip_special_tokens=True)
    
            # Append prediction with no-answer probability for SQuAD v2 metric
            predictions.append({
                "id": str(batch['id'][i]),
                "prediction_text": pred_answer,
                "no_answer_probability": float(no_answer_prob[i].item())
            })
    
            # Append the reference (ground truth) answer
            references.append({
                "id": batch['id'][i],
                "answers": {
                    "text": [true_answer],
                    "answer_start": [true_start]
                }
            })
    
    # Compute evaluation metrics using Huggingface `datasets` metric
    metric.add_batch(predictions=predictions, references=references)
    final_score = metric.compute()
    
    print(f"Exact Match (EM): {final_score['exact']}")
    print(f"F1 Score: {final_score['f1']}")
    
    # Log metrics to MLflow
    mlflow.log_metric("em", final_score['exact'])
    mlflow.log_metric("f1", final_score['f1'])
    
    # Prepare example input for MLflow signature
    input_example = {
        "input_ids": input_ids[0].numpy().tolist(),
        "attention_mask": attention_mask[0].numpy().tolist()
    }
    
    # Infer input-output signature for logging the model
    signature = infer_signature(input_example, outputs[0].detach().numpy())

    # Log the trained model to MLflow
    mlflow.pytorch.log_model(model, 
                            artifact_path=model_artifact_name,
                            signature=signature,
                            input_example=input_example
                             )

Training...


 95%|█████████▌| 1353/1424 [2:44:32<08:32,  7.22s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Epoch 1, Loss: 5.4732


 94%|█████████▍| 1341/1424 [2:39:58<08:56,  6.46s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Epoch 2, Loss: 4.5773


100%|█████████▉| 1422/1424 [2:55:42<00:14,  7.36s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
/var/folders/tg/jlvzghzj7px6zbtkv9swtpg40000gn/T/ipykernel_9301/2115840020.py:45: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")


Epoch 3, Loss: 4.1472
Evaluation...


 27%|██▋       | 99/371 [02:20<06:43,  1.48s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned fo

Exact Match (EM): 44.4331983805668
F1 Score: 55.98606668633911


2024/09/30 18:15:16 INFO mlflow.types.utils: MLflow 2.9.0 introduces model signature with new data types for lists and dictionaries. For input such as Dict[str, Union[scalars, List, Dict]], we infer dictionary values types as `List -> Array` and `Dict -> Object`. 
2024/09/30 18:15:16 INFO mlflow.models.utils: We convert input dictionaries to pandas DataFrames such that each key represents a column, collectively constituting a single row of data. If you would like to save data as multiple rows, please convert your data to a pandas DataFrame before passing to input_example.
/Users/denisbotuk/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
